### IMPORTS ###

In [1]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Inicialización y constantes and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key[:8]=='sk-proj-':
    print("La clave de API parece buena")
else:
    print("¿Puede haber un problema con tu clave API? ¡Visita el cuaderno de resolución de problemas!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

La clave de API parece buena


In [3]:
# La clase para representar una Página Web

class Website:
    """
    Una clase de utilidad para representar un sitio web que hemos scrappeado, ahora con enlaces
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "Sin título"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Título de la Web:\n{self.title}\nContenido de la Web:\n{self.text}\n\n"

In [4]:
frog = Website("https://cursos.frogamesformacion.com")
print(frog.get_contents())
frog.links

Título de la Web:
Frogames
Contenido de la Web:
Ir al contenido principal
Frogames
Menú alternativo
Menú
Iniciar sesión
Ganadora del premio 'Enseñanza online de datos y competencias digitales más innovadora de Europa, 2023'
Pasión por
aprender
con los
mejores
En Frogames Formación te ayudamos a convertirte en todo un experto en: Programación de Videojuegos, Inteligencia Artificial, Machine Learning, Desarrollo de Apps, Data Science y mucho más.
Aprende mientras te diviertes
Cursos, Rutas y Suscripciones
Certificados de finalización
Qué encontrarás
dentro
de Frogames
Cursos online y formación de calidad para toda la família
Rutas temáticas
Rutas organizadas para que aprendas paso a paso, subiendo cada escalón e incrementando tus conocimientos adquiridos
Instructores Expertos
Con un equipo de profesionales y expertos en la materia que te acompañará a lo largo de todo el aprendizaje en la plataforma
Certificados blockchain
Títulos verificados por blockchain para cada habilidad que aprenda

['#main-content',
 '/',
 '/',
 '/users/sign_in',
 'https://cursos.frogamesformacion.com/pages/rutas',
 'https://cursos.frogamesformacion.com/pages/instructores',
 'https://cursos.frogamesformacion.com/pages/certificaciones',
 'https://cursos.frogamesformacion.com/collections',
 '/courses/fundamentos-teoria-musical',
 '/courses/fundamentos-teoria-musical',
 '/courses/bandas-sonoras-2',
 '/courses/bandas-sonoras-2',
 '/courses/liderazgo-data-drive',
 '/courses/liderazgo-data-drive',
 '/courses/bandas-sonoras-peliculas-1',
 '/courses/bandas-sonoras-peliculas-1',
 '/courses/ingenieria-llm',
 '/courses/ingenieria-llm',
 '/courses/principios-solid',
 '/courses/principios-solid',
 '/courses/matematicas-ml-2',
 '/courses/matematicas-ml-2',
 '/courses/obsidian',
 '/courses/obsidian',
 '/courses/trigonometria-desde-cero',
 '/courses/trigonometria-desde-cero',
 '/courses/kotlin-desde-cero',
 '/courses/kotlin-desde-cero',
 '/courses/matematicas-ml',
 '/courses/matematicas-ml',
 '/courses/github-an

In [5]:
link_system_prompt = "Se te proporciona una lista de enlaces que se encuentran en una página web. \
Puedes decidir cuáles de los enlaces serían los más relevantes para incluir en un folleto sobre la empresa, \
como enlaces a una página Acerca de, una página de la empresa, las carreras/empleos disponibles o páginas de Cursos/Packs.\n"
link_system_prompt += "Debes responder en JSON como en este ejemplo:"
link_system_prompt += """
{
    "links": [
        {"type": "Pagina Sobre nosotros", "url": "https://url.completa/aqui/va/sobre/nosotros"},
        {"type": "Pagina de Cursos": "url": "https://otra.url.completa/courses"}
    ]
}
"""

In [6]:
print(link_system_prompt)

Se te proporciona una lista de enlaces que se encuentran en una página web. Puedes decidir cuáles de los enlaces serían los más relevantes para incluir en un folleto sobre la empresa, como enlaces a una página Acerca de, una página de la empresa, las carreras/empleos disponibles o páginas de Cursos/Packs.
Debes responder en JSON como en este ejemplo:
{
    "links": [
        {"type": "Pagina Sobre nosotros", "url": "https://url.completa/aqui/va/sobre/nosotros"},
        {"type": "Pagina de Cursos": "url": "https://otra.url.completa/courses"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Aquí hay una lista de enlaces de la página web {website.url} - "
    user_prompt += "Por favor, decide cuáles de estos son enlaces web relevantes para un folleto sobre la empresa. Responde con la URL https completa en formato JSON. \
No incluyas Términos y Condiciones, Privacidad ni enlaces de correo electrónico.\n"
    user_prompt += "Links (puede que algunos sean links relativos):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(frog))

Aquí hay una lista de enlaces de la página web https://cursos.frogamesformacion.com - Por favor, decide cuáles de estos son enlaces web relevantes para un folleto sobre la empresa. Responde con la URL https completa en formato JSON. No incluyas Términos y Condiciones, Privacidad ni enlaces de correo electrónico.
Links (puede que algunos sean links relativos):
#main-content
/
/
/users/sign_in
https://cursos.frogamesformacion.com/pages/rutas
https://cursos.frogamesformacion.com/pages/instructores
https://cursos.frogamesformacion.com/pages/certificaciones
https://cursos.frogamesformacion.com/collections
/courses/fundamentos-teoria-musical
/courses/fundamentos-teoria-musical
/courses/bandas-sonoras-2
/courses/bandas-sonoras-2
/courses/liderazgo-data-drive
/courses/liderazgo-data-drive
/courses/bandas-sonoras-peliculas-1
/courses/bandas-sonoras-peliculas-1
/courses/ingenieria-llm
/courses/ingenieria-llm
/courses/principios-solid
/courses/principios-solid
/courses/matematicas-ml-2
/courses/m

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
anthropic = Website("https://anthropic.com")
anthropic.links

['/',
 '/claude',
 '/claude',
 '/team',
 '/enterprise',
 '/api',
 '/pricing',
 '/research',
 '/company',
 '/careers',
 '/news',
 'https://claude.ai/',
 'https://www.anthropic.com/research#entry:8@1:url',
 'https://www.anthropic.com/claude',
 'https://claude.ai/',
 '/api',
 '/news/3-5-models-and-computer-use',
 '/claude/sonnet',
 '/claude/haiku',
 '/news/claude-for-enterprise',
 '/research/constitutional-ai-harmlessness-from-ai-feedback',
 '/news/core-views-on-ai-safety',
 '/jobs',
 '/',
 '/claude',
 '/api',
 '/team',
 '/pricing',
 '/research',
 '/company',
 '/customers',
 '/news',
 '/careers',
 'mailto:press@anthropic.com',
 'https://support.anthropic.com/',
 'https://status.anthropic.com/',
 '/supported-countries',
 'https://twitter.com/AnthropicAI',
 'https://www.linkedin.com/company/anthropicresearch',
 'https://www.youtube.com/@anthropic-ai',
 '/legal/consumer-terms',
 '/legal/commercial-terms',
 '/legal/privacy',
 '/legal/aup',
 '/responsible-disclosure-policy',
 'https://trust.an

In [11]:
get_links("https://anthropic.com")

{'links': [{'type': 'Pagina Sobre nosotros',
   'url': 'https://www.anthropic.com/company'},
  {'type': 'Pagina de Carreras', 'url': 'https://www.anthropic.com/careers'},
  {'type': 'Pagina de Investigación',
   'url': 'https://www.anthropic.com/research'},
  {'type': 'Pagina de Claude', 'url': 'https://www.anthropic.com/claude'},
  {'type': 'Pagina de API', 'url': 'https://www.anthropic.com/api'}]}

In [12]:
get_links("https://cursos.frogamesformacion.com")

{'links': [{'type': 'Página de Cursos',
   'url': 'https://cursos.frogamesformacion.com/pages/rutas'},
  {'type': 'Página de Instructores',
   'url': 'https://cursos.frogamesformacion.com/pages/instructores'},
  {'type': 'Página de Certificaciones',
   'url': 'https://cursos.frogamesformacion.com/pages/certificaciones'},
  {'type': 'Página de Nuestros Clientes',
   'url': 'https://cursos.frogamesformacion.com/pages/nuestros-clientes'},
  {'type': 'Página para Empresas',
   'url': 'https://cursos.frogamesformacion.com/pages/frogames-para-empresas'},
  {'type': 'Página de Premios',
   'url': 'https://cursos.frogamesformacion.com/pages/premios'},
  {'type': 'Página de Afiliados',
   'url': 'https://cursos.frogamesformacion.com/pages/afiliados'},
  {'type': 'Página Principal', 'url': 'https://cursos.frogamesformacion.com'}]}

In [13]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Links encontrados:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [14]:
print(get_all_details("https://anthropic.com"))

Links encontrados: {'links': [{'type': 'Pagina Acerca de', 'url': 'https://www.anthropic.com/company'}, {'type': 'Pagina de Carreras', 'url': 'https://www.anthropic.com/careers'}, {'type': 'Pagina de Investigación', 'url': 'https://www.anthropic.com/research'}, {'type': 'Pagina de Claude', 'url': 'https://www.anthropic.com/claude'}, {'type': 'Pagina de Api', 'url': 'https://www.anthropic.com/api'}]}
Landing page:
Título de la Web:
Home \ Anthropic
Contenido de la Web:
Claude
Overview
Team
Enterprise
API
Pricing
Research
Company
Careers
News
Try Claude
AI
research
and
products
that put safety at the frontier
Claude.ai
Meet Claude 3.5 Sonnet
Claude 3.5 Sonnet, our most intelligent AI model, is now available.
Talk to Claude
API
Build with Claude
Create AI-powered applications and custom experiences using Claude.
Learn more
Announcements
Introducing computer use, a new Claude 3.5 Sonnet, and Claude 3.5 Haiku
Oct 22, 2024
Model updates
3.5 Sonnet
3.5 Haiku
Our Work
Product
Claude for Enterp

In [15]:
system_prompt = "Eres un asistente que analiza el contenido de varias páginas relevantes del sitio web de una empresa\
y crea un folleto breve sobre la empresa para posibles clientes, inversores y nuevos empleados. Responde en formato Markdown.\
Incluye detalles sobre la cultura de la empresa, los clientes, las carreras/empleos y los cursos/packs para futuros empleos si tienes la información."

# O descomenta las líneas a continuación para obtener un folleto más humorístico: esto demuestra lo fácil que es incorporar el "tono":

# system_prompt = "Eres un asistente que analiza el contenido de varias páginas relevantes del sitio web de una empresa \
# y crea un folleto breve, divertido y gracioso sobre la empresa para posibles clientes, inversores y nuevos empleados. Responde en formato Markdown.\
#Incluye detalles sobre la cultura de la empresa, los clientes y los cursos/packs para futuros empleos si tienes la información."


In [16]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"Estás mirando una empresa llamada: {company_name}\n"
    user_prompt += f"Aquí se encuentra el contenido de su página de inicio y otras páginas relevantes; usa esta información para crear un breve folleto de la empresa en Markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000] # Truncar si tiene más de 20.000 caracteres
    return user_prompt

In [17]:
get_brochure_user_prompt("Anthropic", "https://anthropic.com")

Links encontrados: {'links': [{'type': 'Pagina Acerca de', 'url': 'https://www.anthropic.com/company'}, {'type': 'Pagina de Carreras', 'url': 'https://www.anthropic.com/careers'}, {'type': 'Pagina de Investigación', 'url': 'https://www.anthropic.com/research'}, {'type': 'Pagina de Productos', 'url': 'https://www.anthropic.com/claude'}]}


'Estás mirando una empresa llamada: Anthropic\nAquí se encuentra el contenido de su página de inicio y otras páginas relevantes; usa esta información para crear un breve folleto de la empresa en Markdown.\nLanding page:\nTítulo de la Web:\nHome \\ Anthropic\nContenido de la Web:\nClaude\nOverview\nTeam\nEnterprise\nAPI\nPricing\nResearch\nCompany\nCareers\nNews\nTry Claude\nAI\nresearch\nand\nproducts\nthat put safety at the frontier\nClaude.ai\nMeet Claude 3.5 Sonnet\nClaude 3.5 Sonnet, our most intelligent AI model, is now available.\nTalk to Claude\nAPI\nBuild with Claude\nCreate AI-powered applications and custom experiences using Claude.\nLearn more\nAnnouncements\nIntroducing computer use, a new Claude 3.5 Sonnet, and Claude 3.5 Haiku\nOct 22, 2024\nModel updates\n3.5 Sonnet\n3.5 Haiku\nOur Work\nProduct\nClaude for Enterprise\nSep 4, 2024\nAlignment\n·\nResearch\nConstitutional AI: Harmlessness from AI Feedback\nDec 15, 2022\nAnnouncements\nCore Views on AI Safety: When, Why, Wh

In [18]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [19]:
create_brochure("Anthropic", "https://anthropic.com")

Links encontrados: {'links': [{'type': 'Página Acerca de', 'url': 'https://www.anthropic.com/company'}, {'type': 'Página de Carreras', 'url': 'https://www.anthropic.com/careers'}, {'type': 'Página de Investigación', 'url': 'https://www.anthropic.com/research'}, {'type': 'Página de Pricing', 'url': 'https://www.anthropic.com/pricing'}, {'type': 'Página de Claude', 'url': 'https://www.anthropic.com/claude'}]}


# Folleto de Anthropic

## Sobre la Empresa

### Misión
Anthropic es una empresa de investigación y seguridad en inteligencia artificial con sede en San Francisco, dedicada a construir sistemas de IA fiables, interpretables y manejables. Nuestro objetivo es asegurar que la IA transforme la vida humana de manera positiva, maximizando sus beneficios y minimizando los riesgos.

### Cultura de Empresa
- **Alta Confianza:** Fomentamos un entorno de alta confianza donde se asume la buena fe y se prioriza la honestidad.
- **Colaboración:** Creemos que el trabajo en equipo es esencial para el éxito. Cada miembro aporta su experiencia a proyectos diversos y colabora en la misión común.
- **Pragmatismo:** Optamos por soluciones simples y prácticas, priorizando el impacto tangible de nuestras acciones.
- **Compromiso Global:** Actuamos siempre con la responsabilidad de maximizar los resultados positivos para la humanidad.

## Productos

### Claude
Claude es nuestra innovadora línea de modelos de IA, diseñada para realizar tareas complejas como razonamiento avanzado, análisis de imágenes, generación de código y procesamiento multilingüe. Claude combina integridad con un rendimiento excepcional, garantizandofiabilidad y seguridad en diversas aplicaciones empresariales.

### API de Claude
Desarrollamos aplicaciones personalizadas e integraciones utilizando nuestra API de Claude, permitiendo a las empresas acceder a nuestras capacidades más avanzadas.

## Clientes
Trabajamos con una amplia variedad de clientes, desde empresas hasta organizaciones sin fines de lucro, utilizando la IA para mejorar sus procesos, análisis y toma de decisiones.

## Carreras y Oportunidades

### Únete a Nuestro Equipo
Buscamos individuos apasionados de diversos campos, desde la investigación y la ingeniería hasta el liderazgo de políticas. Ofrecemos un entorno inclusivo donde cada voz es valorada y cada contribución se reconoce.

### Beneficios para Empleados
- **Salud y Bienestar:** Cobertura médica integral, beneficios de fertilidad inclusivos, y 22 semanas de licencia parental.
- **Compensación Competitiva:** Salarios competitivos, paquetes de acciones y planes de jubilación.
- **Ambiente de Trabajo:** Disponibilidad de comidas diarias, estipendios para la educación y un enfoque en el equilibrio entre trabajo y vida personal.

### Proceso de Selección
Nuestro proceso de selección está diseñado para identificar candidatos agradables, independientemente de su experiencia previa en IA. Valoramos la diversidad y la capacidad de cada candidato para contribuir a nuestras metas.

## Investigación

### Innovación en IA
En Anthropic, conducimos investigaciones pioneras en IA, desarrollando modelos avanzados y técnicas de seguridad para mitigar riesgos asociados a la IA. Trabajamos constantemente en investigaciones que tienen un impacto directo en nuestra misión y en el futuro de la IA.

## Contáctanos
Para más información sobre nuestras soluciones de IA, oportunidades de carrera o colaboraciones, visita nuestro sitio web o contáctanos a través de nuestras redes sociales.

---

Anthropic se compromete a liderar la industria de la IA hacia un futuro más seguro y confiable. Únete a nosotros en este emocionante viaje para modelar el futuro de la tecnología.

In [20]:
create_brochure("Frogames Formación", "https://cursos.frogamesformacion.com")

Links encontrados: {'links': [{'type': 'Página de Instructores', 'url': 'https://cursos.frogamesformacion.com/pages/instructores'}, {'type': 'Página de Certificaciones', 'url': 'https://cursos.frogamesformacion.com/pages/certificaciones'}, {'type': 'Nuestros Clientes', 'url': 'https://cursos.frogamesformacion.com/pages/nuestros-clientes'}, {'type': 'Frogames para Empresas', 'url': 'https://cursos.frogamesformacion.com/pages/frogames-para-empresas'}, {'type': 'Página de Premios', 'url': 'https://cursos.frogamesformacion.com/pages/premios'}, {'type': 'Página de Afiliados', 'url': 'https://cursos.frogamesformacion.com/pages/afiliados'}, {'type': 'Página Principal', 'url': 'https://cursos.frogamesformacion.com'}, {'type': 'Página sobre Cursos', 'url': 'https://cursos.frogamesformacion.com/collections'}]}


# Folleto de Frogames Formación

## Acerca de Frogames
Frogames Formación es una plataforma galardonada con el reconocimiento de "Enseñanza online de datos y competencias digitales más innovadora de Europa, 2023". Nos dedicamos a proporcionar cursos en áreas como **Programación de Videojuegos, Inteligencia Artificial, Machine Learning, Desarrollo de Apps y Data Science**, entre otros. Con más de **500.000 estudiantes** formando parte de nuestra comunidad, estamos comprometidos con la calidad y la innovación educativa.

## Nuestra Cultura
En Frogames, promovemos un ambiente de aprendizaje **divertido y colaborativo**. Creemos que aprender puede y debe ser una experiencia emocionante. Nuestros estudiantes disfrutan de un ámbito en el que la curiosidad y la creatividad son parte esencial del proceso educativo. Con un equipo de **instructores expertos** que ayudan y guían a los estudiantes, fomentamos un sentido de comunidad y apoyo.

### Testimonios de Estudiantes
> "Me encanta aprender en Frogames, sus cursos son especializados y realmente prácticos." - Edwyn Mendoza  
> "Estoy agradecida por las herramientas que Frogames me ha brindado, tanto personal como profesionalmente." - Javiera Constanza Vallejos Jara  

## Cursos y Certificaciones
Ofrecemos una **amplia gama de cursos** en distintos niveles, junto con **rutas temáticas** para asegurar un aprendizaje continuo y sistemático. Al finalizar cada curso, los estudiantes reciben **certificados verificados por blockchain**, garantizando su validez y seguridad, perfectos para mejorar su CV y perfil en redes como LinkedIn.

### Cursos Destacados
- **Desarrollo de Videojuegos con Unity 2021**
- **Machine Learning de la A a la Z**
- **Liderazgo Data Drive: Decisiones Estratégicas con Datos**
- **Ingeniería de LLM: Domina la IA Generativa**

## Oportunidades de Carrera
Frogames no solo es lugar para aprender, sino también para crecer profesionalmente. Buscamos personas apasionadas por la educación y la tecnología. Ofrecemos oportunidades para que te unas a nuestro **equipo de instructores** o para convertirte en **afiliado**, con remuneración por cada venta que consigas.

### Únete a Nosotros
Si tienes un interés en el aprendizaje digital y deseas compartir tu conocimiento, considera formar parte de nuestro equipo o conviértete en afiliado.

## Frogames para Empresas
Ofrecemos soluciones de formación personalizadas a empresas, permitiendo mejorar las competencias de sus empleados. Nuestras formaciones están diseñadas para potenciar el talento humano, aumentar la productividad y reducir costos de contratación.

### Ventajas del Programa para Empresas
- Acceso a un catálogo en constante actualización
- Descuentos para grupos
- Certificaciones blockchain para cada participante

## Conclusión
Frogames Formación es el lugar ideal para aprender, crecer y destacar en el competitivo mundo digital. Con una oferta educativa de calidad, un equipo de expertos dedicados y un ambiente de aprendizaje agradable, estamos listos para ayudarte a alcanzar tus metas. ¡Únete a nosotros y empieza tu viaje de aprendizaje hoy!

Para más información, visita nuestra página web en [Frogames Formación](https://frogamesformacion.com). ¡Te esperamos!

In [21]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [22]:
stream_brochure("Anthropic", "https://anthropic.com")

Links encontrados: {'links': [{'type': 'Pagina Acerca de', 'url': 'https://www.anthropic.com/company'}, {'type': 'Pagina de Carreras', 'url': 'https://www.anthropic.com/careers'}, {'type': 'Pagina de Investigación', 'url': 'https://www.anthropic.com/research'}, {'type': 'Pagina de Claude', 'url': 'https://www.anthropic.com/claude'}, {'type': 'Pagina de API', 'url': 'https://www.anthropic.com/api'}, {'type': 'Pagina de Precios', 'url': 'https://www.anthropic.com/pricing'}]}


# Folleto de Anthropic

## ¿Quiénes Somos?

Anthropic es una empresa de investigación y seguridad en inteligencia artificial (IA) ubicada en San Francisco. Nos dedicamos a desarrollar sistemas de IA que sean seguros, interpretables y dirigibles. Con un equipo interdisciplinario que abarca expertos en aprendizaje automático, física, política y negocios, trabajamos juntos para generar investigación y crear sistemas de IA confiables y beneficiosos.

## Nuestra Misión

Creemos que la IA tendrá un impacto vasto en el mundo. Nuestro propósito es construir sistemas en los que las personas puedan confiar y generar investigación sobre las oportunidades y riesgos asociados con la IA. Tratamos la seguridad de la IA como una ciencia sistemática, compartiendo nuestras aprendizajes con la comunidad global.

## Cultura de la Empresa

En Anthropic fomentamos un ambiente de **alta confianza** y colaboración. Valoramos la honestidad, la madurez emocional y la apertura intelectual. Todos los miembros de la empresa comparten un sentido de unidad, trabajando juntos hacia un objetivo común. Nuestra filosofía se basa en hacer lo simple que funcione, mediante enfoques pragmáticos y empíricos.

### Nuestros Valores

- **Aquí por la misión**: Trabajamos para que la IA transformadora ayude a las personas y a la sociedad a prosperar.
- **Confianza inusual**: Fomentamos la honestidad y el respeto mutuo.
- **Un gran equipo**: La colaboración está en el centro de nuestro trabajo.
- **Hacer lo simple que funciona**: Valoramos la efectividad sobre la complejidad.

## Nuestros Productos

### Claude

Claude es nuestro modelo de IA más inteligente, diseñado para realizar tareas complejas de razonamiento, análisis de imágenes, generación de código y procesamiento multilingüe. Con capacidades avanzadas, Claude es una herramienta versátil para individuos y empresas.

### API de Claude

Permite crear aplicaciones impulsadas por IA y experiencias personalizadas. La integración de Claude en los flujos de trabajo de negocios puede transformar la forma en que las empresas operan.

## Clientes

Trabajamos con **organizaciones** en diversos sectores, incluyendo empresas, organizaciones sin fines de lucro y grupos de la sociedad civil, ayudándoles a implementar procesos de IA que aumenten su eficiencia y eficacia.

## Oportunidades de Carrera

**¿Quieres unirte al equipo que hace que la IA sea segura?** En Anthropic buscamos personas con una variedad de habilidades y antecedentes. Ofrecemos roles para investigadores, ingenieros, expertos en políticas y más. Nuestro proceso de selección está diseñado para identificar candidatos que aporten perspectivas únicas.

### Beneficios

- Seguro de salud, dental y de visión integral.
- Programa de licencia parental de 22 semanas.
- Apoyo generoso para salud mental.
- Compensación competitiva y opciones de acciones.
- Estipendios para bienestar y educación.

## Capacitación y Cursos

Anthropic no ofrece programas de pasantías, pero valoramos la diversidad de experiencias y habilidades en nuestro equipo. Estamos abiertos a candidatos que no tengan experiencia previa en aprendizaje automático, y alentamos a la autoformación en áreas relevantes.

## Contáctanos

Si estás interesado en ser parte de nuestro equipo o aprender más sobre cómo la IA puede ser segura y benefactora, visita nuestra página web o síguenos en nuestras redes sociales.

- **Twitter**: [@AnthropicAI](https://twitter.com/AnthropicAI)
- **LinkedIn**: [Anthropic](https://www.linkedin.com/company/anthropic)

---

Anthropic PBC © 2025. Todos los derechos reservados.

In [23]:
stream_brochure("HuggingFace", "https://huggingface.co")

Links encontrados: {'links': [{'type': 'Pagina Acerca de', 'url': 'https://huggingface.co/huggingface'}, {'type': 'Pagina de Empleos', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'Pagina de Modelos', 'url': 'https://huggingface.co/models'}, {'type': 'Pagina de Datasets', 'url': 'https://huggingface.co/datasets'}, {'type': 'Pagina de Cursos', 'url': 'https://huggingface.co/learn'}, {'type': 'Pagina de Precios', 'url': 'https://huggingface.co/pricing'}, {'type': 'Pagina de Documentación', 'url': 'https://huggingface.co/docs'}, {'type': 'Pagina de Blog', 'url': 'https://huggingface.co/blog'}]}


# Folleto de Hugging Face

## **Hugging Face: La comunidad de IA que construye el futuro**

Hugging Face es una plataforma innovadora y colaborativa enfocada en el aprendizaje automático (ML) y la inteligencia artificial (IA). Con más de 50,000 organizaciones utilizando sus servicios, la comunidad de Hugging Face trabaja junta para democratizar el acceso y el desarrollo en IA.

---

### **Nuestra Misión y Cultura**
En Hugging Face, estamos comprometidos con democratizar el aprendizaje automático y fomentar la colaboración dentro de la comunidad AI. Creemos que juntos podemos construir un futuro más brillante a través de la innovación abierta y el uso responsable de la tecnología. Nuestra cultura está centrada en la colaboración, el aprendizaje continuo y la diversidad.

### **Ofrecemos:**
- **Modelos y Datasets:** Acceso a más de 400,000 modelos y 100,000 datasets. Colabora, crea y comparte tus modelos en la plataforma.
- **Espacios:** Crea aplicaciones y demos utilizando potentes herramientas de ML. Permite a los usuarios experimentar y compartir sus trabajos fácilmente.
- **Soluciones Empresariales:** Ofrecemos opciones de implementación y soporte dedicado para organizaciones que buscan escalar sus proyectos de IA con seguridad y efectividad.

### **Clientes Destacados**
Más de 50,000 organizaciones han confiado en Hugging Face, incluidas empresas líderes como:
- Meta
- AWS
- Google
- Microsoft
- Grammarly

### **Carreras y Oportunidades Laborales**
Estamos en búsqueda de talento apasionado. Trabajar en Hugging Face significa unirte a un equipo diverso que enfrenta desafíos emocionantes en el campo de la IA. Ofrecemos posiciones en diferentes disciplinas, desde desarrollo de software hasta investigación en IA. 

**Ver nuestras oportunidades disponibles en la página de [empleos de Hugging Face](https://huggingface.co/jobs).**

---

### **Formación y Cursos**
Hugging Face facilita un aprendizaje constante a través de cursos en diversas áreas:
- **Curso de NLP:** Introducción a la Procesamiento de Lenguaje Natural utilizando nuestras bibliotecas.
- **Curso de Deep RL:** Aprendizaje profundo en refuerzo.
- **Curso de Visión por Computadora:** Conceptos y aplicaciones en visión por computadora.
- **Curso de Audio:** Uso de transformadores para datos de audio.
- **Cookbook de AI de código abierto:** Notebooks colaborativos para aprender y crear con IA.
  
¡Únete a nuestra comunidad y empieza a aprender sobre ML hoy mismo!

---

### **Planes y Precios**
Hugging Face ofrece varias opciones para adaptarse a tus necesidades:

- **Gratuito:** Acceso a modelos y datasets, y colaboración en la comunidad.
- **Pro ($9/mes):** Características avanzadas, como tasas de límite más altas y acceso anticipado a funciones.
- **Enterprise ($20/usuario al mes):** Soluciones a medida con soporte priorizado y seguridad avanzada.

---

### **Únete a Nosotros**
Si te apasiona la inteligencia artificial y quieres marcar la diferencia, ¡Hugging Face está buscando personas como tú! Explora las oportunidades de empleo y forma parte de nuestra comunidad innovadora.

**Visítanos en:** [huggingface.co](https://huggingface.co)

**Síguenos en redes sociales:**
- [Twitter](https://twitter.com/huggingface)
- [LinkedIn](https://www.linkedin.com/company/huggingface)
- [Discord](https://discord.gg/huggingface)

--- 

Hugging Face, transformando la inteligencia artificial en una comunidad.

In [24]:
stream_brochure("Frogames Formación", "https://cursos.frogamesformacion.com")

Links encontrados: {'links': [{'type': 'Pagina Acerca de', 'url': 'https://cursos.frogamesformacion.com/pages/instructores'}, {'type': 'Pagina de Certificaciones', 'url': 'https://cursos.frogamesformacion.com/pages/certificaciones'}, {'type': 'Pagina de Clientes', 'url': 'https://cursos.frogamesformacion.com/pages/nuestros-clientes'}, {'type': 'Pagina de Premios', 'url': 'https://cursos.frogamesformacion.com/pages/premios'}, {'type': 'Pagina de Afiliados', 'url': 'https://cursos.frogamesformacion.com/pages/afiliados'}, {'type': 'Pagina de Cursos', 'url': 'https://cursos.frogamesformacion.com/collections'}, {'type': 'Pagina de Rutas', 'url': 'https://cursos.frogamesformacion.com/pages/rutas'}]}


# FroGames Formación

## ¡Bienvenido a FroGames!

### Aprende con los Mejores
FroGames Formación es una plataforma premiada en 2023 como la **“Enseñanza online de datos y competencias digitales más innovadora de Europa”**. Nos dedicamos a ayudar a nuestros estudiantes a convertirse en expertos en diversas disciplinas, incluidas Programación de Videojuegos, Inteligencia Artificial, Machine Learning, Desarrollo de Apps y Data Science.

### Lo Que Ofrecemos
- **Cursos Online**: Formaciones de calidad para toda la familia.
- **Rutas Temáticas**: Aprendizaje estructurado para que avances paso a paso.
- **Instructores Expertos**: Un equipo profesional que te acompaña en tu aprendizaje.
- **Certificados por Blockchain**: Verifica tus habilidades adquiridas y compártelos en redes sociales.

### Nuestra Comunidad
Más de **500.000 estudiantes** en todo el mundo confían en nosotros para desarrollar sus habilidades. Testimonios como los de Edwyn Mendoza o Javiera Constanza Vallejos destacan la calidad y el ambiente de aprendizaje en nuestra plataforma.

### Ofertas de Empleo y Oportunidades
FroGames no solo se enfoca en la educación, sino que también busca empoderar a nuevos talentos. Ofrecemos un programa de afiliados donde puedes ganar hasta un **50% de comisión** en cada venta referida. 

Si tienes una audiencia o gestionas un grupo de personas interesadas en la formación, este programa te permite generar ingresos mientras compartes formaciones de calidad.

### Certificaciones
A través de nuestros cursos, podrás obtener **certificaciones digitales validadas por Blockchain**, que garantizan la integridad de tu currículum, permitiéndote destacar en el mercado laboral.

### Formación para Empresas
Estamos comprometidos con la formación corporativa, ayudando a las empresas a mantener una ventaja competitiva mediante nuestra oferta de cursos diseñados específicamente para el entorno empresarial.

### Un Llamado a la Acción
Si deseas iniciarte en el mundo de la programación y el aprendizaje online, te ofrecemos un **curso gratuito de introducción al trading algorítmico** para que conozcas nuestra plataforma y aproveches nuestras actualizaciones constantes.

¡Empieza a aprender con nosotros hoy y descubre un nuevo mundo de oportunidades!

---

### Conectemos
Para más información sobre nuestros cursos, oportunidades de empleo, o cómo unirte a nuestra comunidad de instructores, visita [Frogames Formación](https://frogames.com).

¡Te esperamos en FroGames! 🌟